# Import

In [1]:
import mailbox
import email

import nltk
nltk.download('stopwords')

# import gensim
import gensim.parsing.preprocessing as preprocessing
import gensim.corpora as corpora
import gensim.models as models
# from gensim.models import Word2Vec
# from gensim.models import LdaMulticore


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


# Data preprocessing

## Extract Email

### Functions: Extract email from .mbox file and .eml file


In [4]:
def extract_emails_from_mbox(mbox_file_name):
  messages=[]
  try:
    mbox = mailbox.mbox(mbox_file_name)
    for message in mbox:
      messages.append(message)
  except FileNotFoundError:
    print(f"File not found: {mbox_file_name}")
  return messages


def extract_email_from_eml(eml_file_name):
  email_message=None
  try:
      with open(eml_file_name, "r") as email_file:
          email_message = email.message_from_file(email_file)
  except FileNotFoundError:
      print(f"File not found: {eml_file_name}")
  except Exception as e:
      print(f"An error occurred: {e}")
  return email_message

#From:https://stackoverflow.com/questions/7166922/extracting-the-body-of-an-email-from-mbox-file-decoding-it-to-plain-text-regard

### Functions: Extract component of email

In [5]:
def extract_sender_email(email_message):
  return email_message.get("From")


def extract_subject_email(email_message):
  return email_message.get("Subject")


def extract_content_email(email_message):
  body = None
  if(email_message.is_multipart()):
    for part in email_message.walk():
      if(part.is_multipart()):
        for subpart in part.walk():
          if(subpart.get_content_type() == "text/plain"):
            body = subpart.get_payload(decode=True)
          # elif(subpart.get_content_type() == "text/html"):
          #   body = subpart.get_payload(decode=True)
      elif(part.get_content_type() == "text/plain"):
        body = part.get_payload(decode=True)
  else:
    body = email_message.get_payload(decode=True)
  if(body is not None):
    #chuyển dữ liệu dạng byte string sang string (utf-8)
    body=body.decode('utf-8')
  return body

### Process: Extract phishing emails

In [20]:
phishing_message_bodies = []


phishing_messages = extract_emails_from_mbox("/content/emails-enron-legal-mails.mbox")
for message in phishing_messages:
  body=extract_content_email(message)
  if (body is not None and body.strip()):
    phishing_message_bodies.append(body)
    #print(body)
    #print(body).decode('utf-8')

print(len(phishing_message_bodies))
print(len(phishing_messages))

4279
4279


### Process: Extract benign emails

In [7]:
benign_message_bodies = []


benign_messages = extract_emails_from_mbox("/content/emails-enron-ham.mbox")
for message in benign_messages:
  body=extract_content_email(message)
  if (body is not None and body.strip()):
    benign_message_bodies.append(body)

print(len(benign_message_bodies))
print(len(benign_messages))

0
0


### ===TEST===

In [ ]:
msg = extract_email_from_eml("/content/sample1.eml")
messages = extract_emails_from_mbox("/emails-enron-legal-mails.mbox")
if(msg):
  print(extract_sender_email(msg))
  print(extract_subject_email(msg))
  print(extract_content_email(msg))

In [ ]:
print(phishing_messages[0])
print(phishing_message_bodies[0])

In [ ]:
print(benign_messages[0])
print(benign_message_bodies[0])

## Simple preprocessing

### Custom stop words and preprocessing filters

In [47]:
# Custom stop words and preprocessing filters
stopWords = nltk.corpus.stopwords
stopWords = stopWords.words("english")
stopWords.extend(["nbsp", "font", "sans", "serif", "bold", "arial", "verdana", "helvetica", "http", "https", "www", "html", "enron", "margin", "spamassassin"])

def remove_custom_stopwords(p):
    return preprocessing.remove_stopwords(p, stopwords=stopWords)

CUSTOM_FILTERS = [lambda x: x.lower(), preprocessing.strip_tags, preprocessing.strip_punctuation,
                  preprocessing.strip_multiple_whitespaces, preprocessing.strip_numeric, remove_custom_stopwords,
                  preprocessing.remove_stopwords, preprocessing.strip_short, preprocessing.stem_text]


In [46]:
def custom_preprocessing(messages):
  preprocessed_messages = []
  for message in messages:
    preprocessed = preprocessing.preprocess_string(message,filters = CUSTOM_FILTERS)
    if preprocessed and (preprocessed not in preprocessed_messages):
      preprocessed_messages.append(preprocessed)

  return preprocessed_messages

### Preprocess messages

In [39]:
#preprocessing phishing message bodies
phishing_preprocessed = []
phishing_preprocessed =  custom_preprocessing(phishing_message_bodies)

print(len(phishing_preprocessed))

4088


In [48]:
#preprocessing benign message bodies
benign_preprocessed = []
benign_preprocessed =  custom_preprocessing(benign_message_bodies)

print(len(benign_preprocessed))

0


### ===TEST===

In [35]:
#Test
print(phishing_preprocessed[1])

['trade', 'profit', 'allen', 'grigsbi', 'rest', 'desk', 'total', 'view', 'bonu', 'partli', 'attribut', 'trade', 'partli', 'group', 'perform', 'thought', 'minimum', 'market', 'maximum', 'cash', 'equiti', 'mike', 'number', 'adjust', 'minimum', 'market', 'maximum', 'cash', 'equiti', 'given', 'expect', 'speech', 'point', 'phillip']


# Word2Vec Embedding


In [ ]:
# Train the model on all messages
model = models.Word2Vec(phishing_preprocessed + benign_preprocessed, vector_size=100, min_count=1, workers=3, window=5)

#From: https://www.geeksforgeeks.org/python-word-embedding-using-word2vec/

In [ ]:
model.wv.most_similar("dollar", topn=20)

In [ ]:
model.wv["dollar"]

# LDA Topic Modeling

## Init the number of topics

In [14]:
numTopics = 1024

## Create dictionary and corpus

In [15]:
all_message_preprocessed = phishing_preprocessed + benign_preprocessed

dictionary = corpora.Dictionary(all_message_preprocessed)
corpus = [dictionary.doc2bow(text) for text in all_message_preprocessed]

##  Create LDA model

In [ ]:
LDA_model = models.LdaMulticore(corpus=corpus, num_topics=numTopics, id2word=dictionary)

In [ ]:
#Test
# Print keyword for the topics
print(LDA_model.print_topics())

#Doc2Vec

In [ ]:
tagged_data = [models.doc2vec.TaggedDocument(v, [i]) for i, v in enumerate(all_message_preprocessed)]

In [ ]:
#Khởi tạo và huấn luyện trực tiếp
doc2vec_model = models.Doc2Vec(tagged_data, vector_size=20, window=2, min_count=1, workers=4)

#Tách khởi tạo và huấn luyện1
# doc2vec_model = models.Doc2Vec(tagged_data, vector_size=100, min_count=1, epochs=10)
# doc2vec_model.build_vocab(tagged_data)
# doc2vec_model.train(tagged_data, total_examples=doc2vec_model.corpus_count, epochs=doc2vec_model.epochs)

#Classification

In [ ]:
message_bodies = phishing_message_bodies + benign_message_bodies

In [44]:
def get_file_lines(file_path):
  lines=[]
  try:
    with open(file_path, 'r') as file:
      for line in file:
        lines.append(line.strip())
  except FileNotFoundError:
    print(f"File not found: {file_path}")
  return lines

In [49]:
black_list_words = get_file_lines("/content/spam_wordlist.txt")
black_list = custom_preprocessing(black_list_words)

print(len(black_list_words))
print(len(black_list))

582
369
